<a href="https://colab.research.google.com/github/tharunarasu/Indian-Weather-Analysis-and-Forecast/blob/main/Indian%20Weather%20Analysis%20and%20Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # For Linear Algebra
import pandas as pd # To Work With Data
# for visualizations
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime # Time Series analysis.

In [ ]:
df = pd.read_csv("/content/Weather1 .csv")

In [ ]:
df.head()

,Unnamed: 0,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,0,1901,16.66,18.53,23.72,27.66,29.90,30.21,27.60,26.26,26.43,25.38,20.92,17.87
1,1,1902,18.20,19.84,25.06,28.07,30.36,29.85,27.40,27.15,26.01,24.61,20.37,17.35
2,2,1903,17.36,18.75,22.88,27.29,29.53,30.14,27.77,26.38,26.25,24.39,19.83,16.96
3,3,1904,17.17,19.30,23.28,28.24,29.66,28.96,26.62,26.35,26.07,24.67,20.22,17.66
4,4,1905,16.26,17.54,21.96,25.87,30.30,30.48,27.37,26.75,26.28,24.85,21.35,17.42


In [ ]:
df = pd.read_csv("/content/Weather1 .csv", index_col=0)

In [ ]:
df1 = pd.melt(df, id_vars='YEAR', value_vars=df.columns[1:]) ## This will melt the data
df1.head()

,YEAR,variable,value
0,1901,JAN,16.66
1,1902,JAN,18.20
2,1903,JAN,17.36
3,1904,JAN,17.17
4,1905,JAN,16.26


In [ ]:
df1['Date'] = df1['variable'] + ' ' + df1['YEAR'].astype(str)  
df1.loc[:,'Date'] = df1['Date'].apply(lambda x : datetime.strptime(x, '%b %Y')) ## Converting String to datetime object
df1.head()

,YEAR,variable,value,Date
0,1901,JAN,16.66,1901-01-01
1,1902,JAN,18.20,1902-01-01
2,1903,JAN,17.36,1903-01-01
3,1904,JAN,17.17,1904-01-01
4,1905,JAN,16.26,1905-01-01


In [ ]:
df1.columns=['Year', 'Month', 'Temprature', 'Date']
df1.sort_values(by='Date', inplace=True) ## To get the time series right.
fig = go.Figure(layout = go.Layout(yaxis=dict(range=[0, df1['Temprature'].max()+1])))
fig.add_trace(go.Scatter(x=df1['Date'], y=df1['Temprature']), )
fig.update_layout(title='Temprature Throught Timeline:',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.update_layout(xaxis=go.layout.XAxis(
    rangeselector=dict(
        buttons=list([dict(label="Whole View", step="all"),
                      dict(count=1,label="One Year View",step="year",stepmode="todate")                      
                     ])),
        rangeslider=dict(visible=True),type="date")
)
fig.show()

In [ ]:
###On a closer look, by clicking on One Year View, we can see that the graph seems distorted because this is how the values really are. The temperature varies every year with months.

##May 1921 has been the hottest month in india in the history. What could be the reason ?
#Dec, Jan and Feb are the coldest months. One could group them together as “Winter”.
#Apr, May, Jun, July and Aug are the hottest months. One could group them together as “Summer”.

In [ ]:
fig = px.box(df1, 'Month', 'Temprature')
fig.update_layout(title='Warmest, Coldest and Median Monthly Tempratue.')
fig.show()

In [ ]:
#Insights:
#January has the coldest Days in an Year.
#May has the hottest days in an Year.
#July is the month with least Standard Deviation which means, 
#temperature in July vary least. We can expect any day in July to be a warm day.

In [ ]:
from sklearn.cluster import KMeans
sse = []
target = df1['Temprature'].to_numpy().reshape(-1,1)
num_clusters = list(range(1, 10))

for k in num_clusters:
    km = KMeans(n_clusters=k)
    km.fit(target)
    sse.append(km.inertia_)

fig = go.Figure(data=[
    go.Scatter(x = num_clusters, y=sse, mode='lines'),
    go.Scatter(x = num_clusters, y=sse, mode='markers')
])

fig.update_layout(title="Evaluation on number of clusters:",
                 xaxis_title = "Number of Clusters:",
                 yaxis_title = "Sum of Squared Distance",
                 showlegend=False)
fig.show()

In [ ]:
# Sum of Squared Error is calculated.
#Using Elbow Criterion optimized K is choSen
#As seen in the graph cluster size 3 seems good.

In [ ]:
km = KMeans(3)
km.fit(df1['Temprature'].to_numpy().reshape(-1,1))
df1.loc[:,'Temp Labels'] = km.labels_
fig = px.scatter(df1, 'Date', 'Temprature', color='Temp Labels')
fig.update_layout(title = "Temprature clusters.",
                 xaxis_title="Date", yaxis_title="Temprature")
fig.show()

In [ ]:
#Insights:
#Despite having 4 seasons we can see 3 main clusters based on temperatures.
#Jan, Feb and Dec are the coldest months.
#Apr, May, Jun, Jul, Aug and Sep; all have hotter temperatures.
#Mar, Oct and Nov are the months that have temperatures neither too hot nor too cold.

In [ ]:
fig = px.histogram(x=df1['Temprature'], nbins=200, histnorm='density')
fig.update_layout(title='Frequency chart of temprature readings:',
                 xaxis_title='Temprature', yaxis_title='Count')

In [ ]:
df['Yearly Mean'] = df.iloc[:,1:].mean(axis=1) ## Axis 1 for row wise and axis 0 for columns.
fig = go.Figure(data=[
    go.Scatter(name='Yearly Tempratures' , x=df['YEAR'], y=df['Yearly Mean'], mode='lines'),
    go.Scatter(name='Yearly Tempratures' , x=df['YEAR'], y=df['Yearly Mean'], mode='markers')
])
fig.update_layout(title='Yearly Mean Temprature :',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.show()

Insights
The yearly mean temperature was not increasing till 1980. It was only after 1979 that we can see the gradual increase in yearly mean temperature.
After 2015, yearly temperature has increased drastically.
But, There are some problems in this figure.
We are seeing a monthly like up-down pattern in yearly temperatures as well.
This is not understandable. Because with months, we have a phenomena of seasons and the earth the revolving around sun in a elliptic path. But this pattern is not expected in yearly temperature.

In [ ]:
fig = px.line(df1, 'Year', 'Temprature', facet_col='Month', facet_col_wrap=4)
fig.update_layout(title='Monthly temprature throught history:')
fig.show()


In [ ]:
df['Winter'] = df[['DEC', 'JAN', 'FEB']].mean(axis=1)
df['Summer'] = df[['MAR', 'APR', 'MAY']].mean(axis=1)
df['Monsoon'] = df[['JUN', 'JUL', 'AUG', 'SEP']].mean(axis=1)
df['Autumn'] = df[['OCT', 'NOV']].mean(axis=1)
seasonal_df = df[['YEAR', 'Winter', 'Summer', 'Monsoon', 'Autumn']]
seasonal_df = pd.melt(seasonal_df, id_vars='YEAR', value_vars=seasonal_df.columns[1:])
seasonal_df.columns=['Year', 'Season', 'Temprature']

fig = px.scatter(seasonal_df, 'Year', 'Temprature', facet_col='Season', facet_col_wrap=2, trendline='ols')
fig.update_layout(title='Seasonal mean tempratures throught years:')
fig.show()

In [ ]:
px.scatter(df1, 'Month', 'Temprature', size='Temprature', animation_frame='Year')

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score 

df2 = df1[['Year', 'Month', 'Temprature']].copy()
df2 = pd.get_dummies(df2)
y = df2[['Temprature']]
x = df2.drop(columns='Temprature')

dtr = DecisionTreeRegressor()
train_x, test_x, train_y, test_y = train_test_split(x,y,test_size=0.3)
dtr.fit(train_x, train_y)
pred = dtr.predict(test_x)
r2_score(test_y, pred)

0.9613151305618468

In [ ]:
next_Year = df1[df1['Year']==2021][['Year', 'Month']]
next_Year.Year.replace(2021,2022, inplace=True)
next_Year= pd.get_dummies(next_Year)
temp_2022 = dtr.predict(next_Year)

temp_2022 = {'Month':df1['Month'].unique(), 'Temprature':temp_2022}
temp_2022=pd.DataFrame(temp_2022)
temp_2022['Year'] = 2022
temp_2022

,Month,Temprature,Year
0,JAN,19.90,2022
1,FEB,22.55,2022
2,MAR,26.30,2022
3,APR,28.42,2022
4,MAY,37.50,2022
5,JUN,29.04,2022
6,JUL,28.52,2022
7,AUG,27.88,2022
8,SEP,28.78,2022
9,OCT,26.03,2022


In [ ]:
forecasted_temp = pd.concat([df1,temp_2022], sort=False).groupby(by='Year')['Temprature'].mean().reset_index()
fig = go.Figure(data=[
    go.Scatter(name='Yearly Mean Temprature', x=forecasted_temp['Year'], y=forecasted_temp['Temprature'], mode='lines'),
    go.Scatter(name='Yearly Mean Temprature', x=forecasted_temp ['Year'], y=forecasted_temp['Temprature'], mode='markers')
])
fig.update_layout(title='Forecasted Temprature:',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.show()